In [1]:
import pandas as pd
import numpy as np
import codecs
import os as os
import matplotlib.pyplot as plt
import datetime
from pandas import Series, DataFrame
import pyper
import time
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from math import sqrt

import scipy.spatial.distance as distance
from scipy.cluster.hierarchy import linkage, dendrogram

from sklearn.linear_model import LogisticRegressionCV


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from __future__ import print_function
from IPython.html.widgets import interact, interactive, fixed
from IPython.html import widgets

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
import ipywidgets

In [4]:
TEST_SIZE = 0.2
RANDOM_STATE = 0

In [5]:
with codecs.open("./170117.csv", "r", "utf-8", "ignore") as file3:
    checkdf4 = pd.read_csv(file3, sep=",").set_index("ID")

In [6]:
with codecs.open("./170117checkdf3.csv", "r", "utf-8", "ignore") as file4 :
    checkdf3 = pd.read_csv(file4, sep=",").set_index("Unnamed: 0")

In [15]:
i = 0
for str in  checkdf4.columns  :
    print(str)
    i+= 1
    if i > 20 :
        break
        

初回検査日齢
契約の更新
子どもNo
得点平均
検査回数
検査時の日齢
検査間隔
知覚的発達
知覚的発達の伸び
社会的発達
社会的発達の伸び
言語的発達
言語的発達の伸び
身体的発達
身体的発達の伸び
b_DTW_O_MAX_101
b_DTW_O_MAX_102
b_DTW_O_MAX_103
b_DTW_O_MAX_104
b_DTW_O_MAX_105
b_DTW_O_MAX_106


In [16]:
##卒論に載せたやつ

fig = plt.figure(figsize=(9, 5))
plt.xlim(0, 500)
plt.ylim(0, 48)

for ID in checkdf4.loc[checkdf4.b_EUCL_A_N_MIN_200_5 == True].index :
    data = checkdf3.loc[checkdf3.ID == ID, ["身体的発達", "検査時の日齢"]]
    plt.plot(data.検査時の日齢, data.身体的発達)

In [17]:
data = checkdf3.loc[checkdf3.ID == 1098802, ["身体的発達", "検査時の日齢"]]

In [18]:
data["身体的発達"]

Unnamed: 0
21      11.0
571     15.0
1016    18.0
1499    18.0
1997    21.0
2495    22.0
2741    26.0
Name: 身体的発達, dtype: float64

In [ ]:
fig = plt.figure(figsize=(9, 5))
plt.xlim(0, 500)
plt.ylim(0, 48)

for ID in checkdf4.loc[checkdf4.k_EUCL_B_N_AVE_200_14 == True].index :
    data = checkdf3.loc[checkdf3.ID == ID, ["知覚的発達", "検査時の日齢"]]
    plt.plot(data.検査時の日齢, data.知覚的発達)


In [ ]:
%matplotlib inline



fig = plt.figure(figsize=(12, 9))
plt.xlim(0, 500)
plt.ylim(0, 25)

for ID in checkdf4.loc[checkdf4.k_EUCL_A_N_MIN_100_4 == True].index :
    data = checkdf3.loc[checkdf3.ID == ID, ["知覚的発達", "検査時の日齢"]]
    plt.plot(data.検査時の日齢, data.知覚的発達)

In [ ]:
checkdf4.loc[checkdf4.b_EUCL_B_N_AVE_200_10 == True]

In [ ]:
checkdf4.loc[checkdf4.k_EUCL_B_N_AVE_200_14 == True]

In [ ]:
#s_EUCL_B_N_AVE_200_
def scores(length, interpolate, normalize, clustMethod, clustSize) :
    
    #length : DTW, EUCL
    #interpolate : O, A, B
    #normalize : "", "N_"
    #clustMethod : MIN, MAX, AVE
    #clustSize : 100, 200


    labels = ["子どもNo", "初回検査日齢", "検査時の日齢", "検査間隔", "検査回数",
              "得点平均", "身体的発達",  "知覚的発達", "言語的発達", "社会的発達"]

    labels.extend([str(x) + "_" + str(length) + "_" + str(interpolate) + "_" + str(normalize) + str(clustMethod) + "_" + str(clustSize) + "_" + str(y)
                   for x in ["b", "k", "l", "s"] for y in range(1, clustSize + 1)])

    X = checkdf4.loc[:, labels]
    y = checkdf4.loc[:, '契約の更新'].replace(False, -1.0)

    X_scaled = preprocessing.scale(X)
    #X_scaled = X
    X_train, X_val, y_train, y_val = train_test_split(X_scaled,
                                                      y,
                                                      test_size=TEST_SIZE,
                                                      random_state=RANDOM_STATE)

    log = LogisticRegressionCV()
    log.fit(X_train, y_train)
    y_val_pred = log.predict(X_val)

#    print(mean_squared_error(y_val, y_val_pred) / 4)
#    print((y_val - y_val_pred).value_counts())
    print(roc_auc_score(y_val, y_val_pred), length, interpolate, normalize, clustMethod, clustSize)
#    print(roc_auc_score(y_val, y_val_pred))
#    print(log.coef_)

In [ ]:
for length in ["DTW"] :
    for interpolate in ["O", "A", "B"] :
        for normalize in ["", "N_"] :
            for method in ["MIN", "MAX", "AVE"] :
                for clustSize in [100, 200] :
                    scores(length, interpolate, normalize, method, clustSize)
                    
for length in ["EUCL"] :
    for interpolate in ["A", "B"] :
        for normalize in ["", "N_"] :
            for method in ["MIN", "MAX", "AVE"] :
                for clustSize in [100, 200] :
                    scores(length, interpolate, normalize, method, clustSize)
#scores("DTW", "O", "N_", "MAX", 100) 

In [ ]:
for length in ["DTW"] :
    for interpolate in ["O", "A", "B"] :
        for normalize in ["", "N_"] :
            for method in ["MIN", "MAX", "AVE"] :
                for clustSize in [10, 20, 50, 100, 200] :
                    scores(length, interpolate, normalize, method, clustSize)
                    
for length in ["EUCL"] :
    for interpolate in ["A", "B"] :
        for normalize in ["", "N_"] :
            for method in ["MIN", "MAX", "AVE"] :
                for clustSize in [10, 20, 50, 100, 200] :
                    scores(length, interpolate, normalize, method, clustSize)



In [ ]:
checkdf3.loc[checkdf3.ID == 2061303, ["月齢", "身体的発達"]]

In [ ]:
fig = plt.figure(figsize=(12, 9))

for label in ["身体的発達", "知覚的発達", "言語的発達", "社会的発達"]:
    data = checkdf3.loc[checkdf3.ID == 2061303]
    plt.plot(data.検査時の日齢, data[label])

In [ ]:
checkdf3.loc[checkdf3.ID == 2061303, ["身体的発達", "知覚的発達", "言語的発達", "社会的発達"]]